In [1]:
import pycoin.ecdsa.secp256k1 as secp256k1
g = secp256k1.secp256k1_generator

## Base point G

In [2]:
x = hex(g.raw_mul(1)[0])
y = hex(g.raw_mul(1)[1])
print('x =', x, '\ny =', y)

x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 
y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8


## 1. Evaluate 4G

In [3]:
(x_4, y_4) = g.raw_mul(4)
print('x =', hex(x_4), '\ny =', hex(y_4))

x = 0xe493dbf1c10d80f3581e4904930b1404cc6c13900ee0758474fa94abe8c4cd13 
y = 0x51ed993ea0d455b75642e2098ea51448d967ae33bfbdfe40cfe97bdc47739922


## 2. Evaluate 5G

In [4]:
(x_5, y_5) = g.raw_mul(5)
print('x =', hex(x_5), '\ny =', hex(y_5))

x = 0x2f8bde4d1a07209355b4a7250a5c5128e88b84bddc619ab7cba8d569b240efe4 
y = 0xd8ac222636e5e3d6d4dba9dda6c9c426f788271bab0d6840dca87d3aa6ac62d6


## 3. Evaluate Q = dG

In [5]:
d = 922002
(x_d, y_d) = g.raw_mul(d)
print('x =', hex(x_d), '\ny =', hex(y_d))

x = 0x603c84e1fafe0356199e5e3f4ad75e5e09edbfd4c9d0b36e3d9b285ac7cac955 
y = 0xdce1b80bcfdd76422ade25eab9a19ae51472677c2059beb0d782990e7fdb9da2


## 4. How many doubles and additions respectively are required to evaluate dG?

In [6]:
d = 922002

binary = bin(d)[2:]
print(d, 'G =', binary, 'G\n')
double = 0
add = 0

number = int(binary[0], 2)
print(number, '\t initial')
for i in range(1, len(binary)):
    if binary[i]=='0':
        number*=2
        double+=1
        print(bin(number)[2:], '\t double')
    elif binary[i]=='1':
        number*=2
        double+=1
        print(bin(number)[2:], '\t double')
        number+=1
        add += 1
        print(bin(number)[2:], '\t add')
print('\ndouble:', double)
print('add:', add)
print('number:', number)

922002 G = 11100001000110010010 G

1 	 initial
10 	 double
11 	 add
110 	 double
111 	 add
1110 	 double
11100 	 double
111000 	 double
1110000 	 double
11100000 	 double
11100001 	 add
111000010 	 double
1110000100 	 double
11100001000 	 double
111000010000 	 double
111000010001 	 add
1110000100010 	 double
1110000100011 	 add
11100001000110 	 double
111000010001100 	 double
1110000100011000 	 double
1110000100011001 	 add
11100001000110010 	 double
111000010001100100 	 double
1110000100011001000 	 double
1110000100011001001 	 add
11100001000110010010 	 double

double: 19
add: 7
number: 922002


## 5.  Try your best to evaluate dG as fast as possible.

In [7]:
d = 922002

binary = bin(d)[2:]
print(d, 'G =', binary, 'G\n')
double = 0
add = 0
sub = 0
pos = 0

number = int(binary[0], 2)
print(number, '\t initial')
for i in range(1, len(binary)):
    if i<pos:
        continue
    if binary[i]=='0':
        number*=2
        double+=1
        print(bin(number)[2:], '\t double')
    elif binary[i]=='1':
        if i+1<len(binary) and binary[i+1]=='1':
            number+=1
            add+=1
            print(bin(number)[2:], '\t add')

            for j in range(i, len(binary)+1):
                if j==len(binary) or binary[j]=='0':
                    number-=1
                    sub+=1
                    print(bin(number)[2:], '\t sub')
                    pos=j
                    break
                elif binary[j]=='1':
                    number*=2
                    double+=1
                    print(bin(number)[2:], '\t double')
        else:
            number*=2
            double+=1
            print(bin(number)[2:], '\t double')
            number+=1
            add += 1
            print(bin(number)[2:], '\t add')
                
print('\ndouble:', double)
print('add:', add)
print('sub:', sub)
print('number:', number)

922002 G = 11100001000110010010 G

1 	 initial
10 	 add
100 	 double
1000 	 double
111 	 sub
1110 	 double
11100 	 double
111000 	 double
1110000 	 double
11100000 	 double
11100001 	 add
111000010 	 double
1110000100 	 double
11100001000 	 double
11100001001 	 add
111000010010 	 double
1110000100100 	 double
1110000100011 	 sub
11100001000110 	 double
111000010001100 	 double
1110000100011000 	 double
1110000100011001 	 add
11100001000110010 	 double
111000010001100100 	 double
1110000100011001000 	 double
1110000100011001001 	 add
11100001000110010010 	 double

double: 19
add: 5
sub: 2
number: 922002


## 6. Sign the transaction with random number k and private key d.

In [8]:
import pycoin.ecdsa.secp256k1 as secp256k1
import random
g = secp256k1.secp256k1_generator

z = 0x38316DC32F31B3BC25DC18A61E682E86837877689209A3EC1562CE59E47CE13B # transaction after hash
n = g.order()
k = random.randint(1, n-1) # cryptographically secure random
d = 922002 # private key

(x_1, y_1) = g.raw_mul(k)
r = x_1 % n
k_inv = g.inverse_mod(k, n)
s = k_inv*(z+r*d) % n
print('r =', hex(r))
print('s =', hex(s))

r = 0xd58997d512a6905308c8aeb080f215c22ba908130f2f518c4943e450ef3353e3
s = 0x8c1f3a483ec78c56fcea98e6a9119b2312f110e4d227ad3eff825bd647e61621


## 7. Verify the digital signature with your public key Q.

In [9]:
import pycoin.ecdsa.secp256k1 as secp256k1
import random
g = secp256k1.secp256k1_generator

z = 0x38316DC32F31B3BC25DC18A61E682E86837877689209A3EC1562CE59E47CE13B # transaction after hash
n = g.order()
d = 922002
r = r
s = s
Q = g.raw_mul(d)

if g.contains_point(Q[0], Q[1]):
    print('valid Q\n')
else:
    print('invalid Q\n')

s_inv = g.inverse_mod(s, n)
u_1 = (z*s_inv) % n
u_2 = (r*s_inv) % n
(x_1, y_1) = g.raw_mul(u_1)+g.multiply(Q, u_2)
if r%n==x_1%n:
    print('valid signature')
else:
    print('invalid signature')
print('    r  mod n =', hex(r%n))
print('   x_1 mod n =', hex(x_1%n))

valid Q

valid signature
    r  mod n = 0xd58997d512a6905308c8aeb080f215c22ba908130f2f518c4943e450ef3353e3
   x_1 mod n = 0xd58997d512a6905308c8aeb080f215c22ba908130f2f518c4943e450ef3353e3
